# Optical Flow-based Tracking


In [11]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture(r"C:\Users\vamsh\Desktop\WAF2\NLP_walking.mp4")


In [12]:
# Take the first frame and convert it into grayscale
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)


In [13]:
# Parameters for Lucas_Kanade optical flow
lk_params = dict(winSize=(15,15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [14]:
ret

True

In [15]:
# Parameters for goodFeaturesToTrack
feature_params = dict(maxCorners = 100,
                      qualityLevel = 0.3, 
                      minDistance = 7,
                      blockSize = 7)

In [16]:
# Create mask for drawing
mask = np.zeros_like(first_frame)

# Find initial points to track
p0 = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    next_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, next_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        x, y = new.ravel()
        a,b = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(x), int(y)), (0, 255, 0), 2 )
        frame = cv2.circle(frame, (int(x), int(y)), 5, (0,0,255), -1)
        
    img = cv2.add(frame, mask)

    cv2.imshow("The image of Optical Flow: " , img)

    p0 = good_new.reshape(-1, 1, 2)
    prev_gray = next_gray # updating the frame to the previous one

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
        
    

In [17]:
cap.release()
cv2.destroyAllWindows()